<html>
<div style="width:100%">
  <div style="width:90%; float:left; background:white; margin:10px">
    <img style ="width:100%"src ="https://raw.githubusercontent.com/publicunpublic/QCMM_Jupyter/main/full.svg">
    <div style="margin-top:2.5%;position:absolute;background-color:#0e4be5;width:6%;height:0.3%"></div>
  </div>
  </div>


<html>
<center>
<br>
<h1 style="color:#E6610F;"> Compute Hessian Matrix for Optimized TS </h1>
<br><br>
</center>

In [1]:
import qcportal as ptl

## Define Variables:

In [2]:
ts_entry_name = 'ts1_02'
username = None
password = None
ts_col_name = "salen-OMe_d-lac_ts"
opt_model_chem = 'pbe0-d3_def2-svp'

In [3]:
client = ptl.FractalClient(address="152.74.10.245:7778", username = username, password = password, verify=False)

## A) Get Optimized Molecule ID 

In [4]:
ds_opt = client.get_collection("OptimizationDataset", ts_col_name)
ds_opt.status(collapse = False)

,pbe0-d3_def2-svp
ts1_01,COMPLETE
ts1_03,COMPLETE
ts1_02,ERROR
ts1_04,INCOMPLETE


In [72]:
rr = ds_opt.get_record(ts_entry_name, specification = opt_model_chem)
mol_opt_ts = rr.get_final_molecule()
mol_opt_ts

NGLWidget()

In [73]:
ts_id = mol_opt_ts.id
ts_id

'44569'

## B) Check RMSD within dataset

In [48]:
recs = ds_opt.data.records
for nmol2 in recs.keys():
    rr2 = ds_opt.get_record(nmol2, specification=opt_model_chem)
    if rr.status == 'COMPLETE' and rr2.status == "COMPLETE":
        mol2 = rr2.get_final_molecule()
        rmsd_val = mol2.align(mol_opt_ts, atoms_map = True, run_mirror=True )[1]['rmsd']
        print(nmol1 + ': ' + opt_model_chem + ' ' + opt_model_chem + ' ' + str(rmsd_val))

ts1_03: pbe0-d3_def2-svp pbe0-d3_def2-svp 1.5790646885908388
ts1_03: pbe0-d3_def2-svp pbe0-d3_def2-svp 0.0


## C) Send Hessian Computation 

In [74]:
#Send the computation to the client

manager_tag = 'tera'

kw = ptl.models.KeywordSet(id = 1, **{"values": {"dftd": "d3", "dftgrid" : 2, "convthre" : "3.0e-7", "threall": "1.0e-13", "scf" : "diis+a"}})
kw_id = client.add_keywords([kw])[0]

method, basis = opt_model_chem.split('_')

if "d3" in method:
    method = method.split('-')[0]

rc = client.add_compute(program = "terachem", 
                       method = method, 
                       basis = basis, 
                       driver = "hessian", 
                       molecule = ts_id, 
                       tag= manager_tag,
                       keywords = kw_id,
                       full_return = True)
rc

TaskQueuePOSTResponse(meta=ResponsePOSTMeta(errors=[], success=True, error_description='False', n_inserted=1, duplicates=[], validation_errors=[]), data=<ComputeResponse(nsubmitted=1 nexisting=0)>)

## C) Check Hessian Computation 

You need to run the above line first to check the state of the computation. If you already ran it once it won't run it again, just get the job id. 

In [75]:
#  Check Hessian job status
cr = rc.data
ids = cr.dict()['ids']
rr_h = client.query_results(ids)[0]
rr_h.dict()['status']

<RecordStatusEnum.incomplete: 'INCOMPLETE'>